# Elimination

This section will introduce the idea of elimination to easily solve linear systems of equations. We will be working primarily with the matrix-vector form of linear systems $A \mathbf x = \mathbf b$.



## Row Echelon Form

Our goal is to transform general linear systems into upper triangular systems. It is easier to do so using the $A \mathbf x = \mathbf b$ form of the system. A system of linear equations is upper triangular if the coefficient matrix $A$ is written in **row echelon form (REF)**. The conditions are as follows:

* The first nonzero entry in each row, called a **pivot** entry (a.k.a. **leading entry**), is strictly to the right of the pivot in the preceding row.
* All zero rows are at the bottom of the matrix.

The matrix below, assuming $A_{1,1}$, $A_{2,3}$, and $A_{2,4}$ are non-zero, is an example of a matrix in row echelon form. Note that the matrix may be of any size. 

$$ \begin{bmatrix}
A_{1,1} & A_{1,2} & A_{1,3} & A_{1,4} \\
0 & 0 & A_{2,3} & A_{2,4} \\
0 & 0 & 0 & A_{2,4} \\
0 & 0 & 0 & 0
\end{bmatrix} $$

In the example above, $A_{1,1}$, $A_{2,3}$, and $A_{2,4}$ are the pivot entries. The columns that contain the pivot entries are called **pivot columns**, and the corresponding variables ($x_1$, $x_3$, $x_4$) are called **pivot variables** (a.k.a. **basic variables**). All other variables are called **free variables**.

### Reduced row echelon form

A special case of row echelon form is **reduced row echelon form (RREF)**. In addition to the conditions above, a matrix is in reduced row echelon form if the following conditions hold:

* All pivot entries are equal to 1.
* Pivot columns contain only one nonzero entry--the pivot entry.

The matrix above is not in RREF, but the following one is:

$$ \begin{bmatrix}
1 & A_{1,2} & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0
\end{bmatrix} $$

Any linear system with a coefficient matrix in (reduced) row echelon form is "upper triangular", and can hence be solved easily using "back substitution".

## Row Operations

The transformation of a matrix into (reduced) row echelon form can be done using one of three *row operations*. Importantly, these operations do not change the solution set of the original system of equations. They are as follows:

* Multiply a row by a nonzero scalar.
* Replace a row by the sum of it with another row.
* Swap two rows.

Given two equations, the first two operations can be combined to *eliminate* a particular matrix entry to zero. By systematically performing these operations and swapping rows when necessary, a matrix can be transformed into (reduced) row echelon form. 

As an example, take the following matrix:

$$ \begin{bmatrix}
A_{1,1} & A_{1,2} & \cdots & A_{1,n} \\
A_{2,1} & A_{2,2} & \cdots & A_{2,n} \\
0 & A_{3,2} & \cdots & A_{3,n} 
\end{bmatrix} $$

To transform this into row echelon form, we need to eliminate $A_{2,1}$ and $A_{3,2}$. Here are the row operations to do so:

* Replace row 2 with (row 2 $- \frac{A_{2,1}}{A_{1,1}} \times $ row 1).
* Replace row 3 with (row 3 $- \frac{A_{3,2}}{A_{2,2} - \frac{A_{2,1}}{A_{1,1}}A_{1,2}} \times $ row 2).

Performing these operations results in the following matrix:

$$ \begin{bmatrix}
A_{1,1} & A_{1,2} & \cdots & A_{1,n} \\
0 & A_{2,2} - \frac{A_{2,1}}{A_{1,1}} A_{1,2} & \cdots & A_{2,n} - \frac{A_{2,1}}{A_{1,1}} A_{1,n} \\
0 & 0 & \cdots & A_{3,n} - \left(\frac{A_{3,2}}{A_{2,2} - \frac{A_{2,1}}{A_{1,1}}A_{1,2}}\right) (A_{2,n} - \frac{A_{2,1}}{A_{1,1}} A_{1,n})
\end{bmatrix} $$

While not pretty, this matrix is certainly in row echelon form. As a variation, suppose that $A_{1,1} = 0$ to start with, while $A_{2,1} \neq 0$. In order for all pivot entries to be ordered correctly, we would first swap rows 1 and 2. Then the row operations above can be applied to the resultant matrix.


## Elimination

We are now ready to present the elimination algorithm, which transforms a matrix into row echelon form. 

**Outer loop**: We process one row at a time, keeping track of the current row using variable `pr`. The pivot column is indicated using `pc`; both indices start at 0, but `pc` is incremented separately from `pr` as soon as we find a column without a pivot. The algorithm ends when we reach either the last row or last column.

**Possible row swap**: At the beginning of each iteration, we need to make sure that the current entry in `A[pr, pc]` is nonzero. We use the `np.nonzero` procedure to return the indices of nonzero entries at or below row `pr` in the column `pc`. If the returned result is empty, there is no nonzero entry, and we move on to the next column by incrementing `pc`. Otherwise, we perform a row swap with the found row (which may also just be `pr` itself).

**Eliminate non-zeros below the pivot**: The last step is to row reduce each row below row `pr`. From row `i`, we subtract row `pr` scaled by `A[i,pc]/A[pr,pc]`. This is guaranteed to make the `A[i,pc]` entry 0. While we can use a loop to do this reduction for each row, there is a cleverer way to do this all at once using the *outer product*. 

A quick word on the *outer product.* Recall from lecture that the outer product of an $m$-vector $\mathbf{v} = [v_1, \dots, v_m]$ and an $n$-vector $\mathbf{u} = [u_1, \dots, u_n]$ is the $m \times n$ matrix that we get from taking matrix product of $m \times 1$ vector $\mathbf{v}$ and $1 \times n$ vector $\mathbf{u}$:
$$\mathbf{v} \mathbf{u}^T = \begin{bmatrix} v_1 \\ \vdots \\ v_m \end{bmatrix} \begin{bmatrix} u_1 & \dots & u_n \end{bmatrix} = \begin{bmatrix} 
v_1u_1 & v_1u_2 & \dots & v_1u_n \\
v_2u_1 & \dots & \dots & v_2u_n \\
\vdots & \ddots & \ddots & \vdots \\
v_nu_1 & \dots & \dots & v_n u_n
\end{bmatrix}.$$

The outer product between `A[pr+1:, pc]/A[pr,pc]` and `A[pr]` is an outer product between an $m - (pr + 1)$ vector and an `n` vector, forming an $(m - (pr + 1)) \times n$ matrix. Because this matrix matches the shape of `A[pr+1:]`, we can immedaitely just subtract via matrix-matrix subtraction to perform the row eliminations at once. See the section with $A$ below for an example of this.

Note: The function `REF` below only transforms the matrix to Row Echelon Form (REF), not Reduced Row Echelon Form (RREF). But REF is enough for the purposes described below.

In [1]:
import numpy as np

def REF(A):
  A = A.astype(float)
  pr = 0    # row with current pivot
  pc = 0    # col with current pivot
  
  while pr < A.shape[0] and pc < A.shape[1]:  # run until last row or last column
    nonzeros = np.nonzero(A[pr:,pc])[0]       # find nonzero entries in column
    if nonzeros.size == 0:                    # no nonzero entries, move to next column
      pc += 1

    else:
      row = nonzeros[0] + pr                  # if current row nonzero, row = pr
      A[[pr, row]] = A[[row, pr]]             # swap rows with the found pivot
      scaled = np.outer(A[pr+1:,pc]/A[pr,pc], A[pr])
      # numpy trick to do the elimination
      
      A[pr+1:] -= scaled                      # row-reduce all rows below pivot
      pr += 1                                 # move to next pivot position
      pc += 1
  
  return A

## Example

Suppose we are running `REF` on the following matrix:

$$ A = \begin{bmatrix} 
0 & 3 & -6 & 6 & 4 \\ 3 & -7 & 8 & -5 & 8 \\ 
3 & -9 & 12 & -9 & 6 \end{bmatrix} $$

`pr` and `pc` are both initialized to 0. we look downward for a nonzero pivot entry starting from `A[0,0]` and find that `A[1,0]` is the first one. We swap rows `0` and `1` and obtain

$$ \begin{bmatrix} 
3 & -7 & 8 & -5 & 8 \\ 0 & 3 & -6 & 6 & 4 \\
3 & -9 & 12 & -9 & 6 \end{bmatrix} .$$

The next step is to row reduce all rows below row `pr` so that all entries in the `pc` column are 0. In this example, this means replacing row 3 with row 3 minus 1 times row 1. Row 2 already has a 0 in the `pc` column.

$$ \begin{bmatrix} 
3 & -7 & 8 & -5 & 8 \\ 0 & 3 & -6 & 6 & 4 \\
0 & -2 & 4 & -4 & -2 \end{bmatrix} $$

As an example of the outer product implementation, we instead subtract the $(m - (pr + 1)) \times n$ matrix `np.outer(A[pr+1:,pc]/A[pr, pc], A[pr])` from the matrix `A[pr+1:]`. Both of these matrices are $2 \times 5$. For clarity, the subtraction is shown below:

$$
\begin{bmatrix}
0 & 3 & -6 & 6 & 4 \\
3 & -9 & 12 & -9 & 6
\end{bmatrix}
-
\begin{bmatrix}
0 & 0 & 0 & 0 & 0 \\
3 & -7 & 8 & -5 & 8
\end{bmatrix}
=
\begin{bmatrix}
0 & 3 & -6 & 6 & 4 \\
0 & -2 & 4 & -4 & -2
\end{bmatrix}.
$$

Notice that the above $2 \times 5$ matrix is exactly the result of doing the elimination on the second and third rows.

We increment both `pr` and `pc` to 1. `A[pr,pc]` is nonzero so it is a pivot entry. We eliminate all entries below `A[pr,pc]`; here, this is just row 3. We replace row 3 with row 3 minus $-\frac{2}{3}$ times row 2.

$$ \begin{bmatrix} 
3 & -7 & 8 & -5 & 8 \\ 0 & 3 & -6 & 6 & 4 \\
0 & 0 & 0 & 0 & 2/3 \end{bmatrix} $$

At this point, the algorithm is nearly finished; you should see that the matrix above is already in row echelon form. `pr` and `pc` are incremented to 2 for the next iteration. `A[pr,pc]` is 0 and there are no rows below so `pc` is incremented by 1. Same thing happens with the new `A[pr,pc]` entry. Now `A[pr,pc] = A[2,4]` is nonzero, but there is no reduction to be done because there are no rows below the last one. The algorithm finishes at this point.


In [2]:
A = np.array([[0, 3, -6, 6, 4], [3, -7, 8, -5, 8], [3, -9, 12, -9, 6]])
print(A)

[[ 0  3 -6  6  4]
 [ 3 -7  8 -5  8]
 [ 3 -9 12 -9  6]]


In [3]:
print(REF(A))

[[ 3.         -7.          8.         -5.          8.        ]
 [ 0.          3.         -6.          6.          4.        ]
 [ 0.          0.          0.          0.          0.66666667]]


# Solutions 

It is possible for a linear system to have one, multiple, or no solutions. We first introduce the notion of augmented matrices, and then we describe when each case occurs. 

## Augmented Matrices

Performing row operations on a coefficient matrix $A$ is equivalent to performing the same operations on the LHS of the corresponding linear system. Since each row corresponds to an equation, the same operations must also be applied to the RHS (the $b_i$ values). Given a linear system of equations, we can thus form the **augmented matrix** 

$$\begin{bmatrix} A & \mathbf b \end{bmatrix},$$

which is just the original coefficient matrix with the RHS appended as an additional column. Any row operations performed on this matrix correspond to the same operations on the entire linear system. Once this matrix is in row echelon form, the solution set can be readily extracted using back substitution.

## Solution Cases

Suppose the augmented matrix above is in row echelon form. The conditions for solutions are as follows:

### No solutions

There are no solutions to the linear system if there is any row of the form 

$$ \begin{bmatrix} 0 & 0 & \cdots & 0 & c \end{bmatrix} $$

where $c \neq 0$. This is equivalent to an equation $0 = c$, which is inconsistent.

### One solution

There is one (unique) solution if every column (except for the last column) contains a pivot entry. Equivalently, there are no free variables; every variable is a pivot variable.

### Infinitely many solutions

There are infinitely many solutions if there is at least one free variable; at least one column does not contain a pivot entry. One way to think about this is that the free variables can be assigned arbitrary values, after which the pivot variables can be uniquely solved. Since there are infinitely many possibilities for free variable assignments, there are infinitely many solutions to the original linear system.

In [4]:
# Example 1: Unique solution, each column contains pivot
Ab = np.array([[1,-2,1,0],[0,2,-8,8],[5,0,-5,10]])
print(Ab)
print(REF(Ab))

[[ 1 -2  1  0]
 [ 0  2 -8  8]
 [ 5  0 -5 10]]
[[  1.  -2.   1.   0.]
 [  0.   2.  -8.   8.]
 [  0.   0.  30. -30.]]


In [5]:
# Example 2: No solution, last row has the form [0 c]
Ab = np.array([[0,1,-4,8],[2,-3,2,1],[4,-8,12,1]])
print(Ab)
print(REF(Ab))

[[ 0  1 -4  8]
 [ 2 -3  2  1]
 [ 4 -8 12  1]]
[[ 2. -3.  2.  1.]
 [ 0.  1. -4.  8.]
 [ 0.  0.  0. 15.]]


In [6]:
# Example 3: Multiple solutions, x3 is free
Ab = np.array([[0,-3,-6,4,9],[-1,-2,-1,3,1],[-2,-3,0,3,-1],[1,4,5,-9,-7]])
print(Ab)
print(REF(Ab))

[[ 0 -3 -6  4  9]
 [-1 -2 -1  3  1]
 [-2 -3  0  3 -1]
 [ 1  4  5 -9 -7]]
[[-1.         -2.         -1.          3.          1.        ]
 [ 0.         -3.         -6.          4.          9.        ]
 [ 0.          0.          0.         -1.66666667  0.        ]
 [ 0.          0.          0.          0.          0.        ]]


# Elimination Using Matrices

This last section will present a different way of looking at elimination: matrix multiplication.

## Elementary Matrices

Each of the row operations can be represented by a particular $m \times m$ **elementary matrix** $E$, and the application of a row operation to a matrix $A$ is equivalent to multiplying $EA$.

Scaling row $i$ of $A$ can be achieved using a *scaling matrix* $E^{\textrm{scale}}_i$, which looks almost like the identity matrix $I_m$, with the $(i,i)$ entry equal to the scaling factor instead of $1$.

The replacement of row $i$ by (row $i$ minus $l_{ij}$ times row $j$) is achieved using an *elimination matrix* $E^{\textrm{elim}}_{ij}$. This matrix looks almost like the identity matrix $I_m$, in addition to the $(i,j)$ entry being equal to $-l_{ij}$.

Swapping rows $i$ and $j$ is achieved using a *permutation matrix* $E^{\textrm{perm}}_{ij}$. Again starting with the identity matrix $I_m$, $E^{\textrm{perm}}_{ij}$ may be formed by swapping rows $i$ and $j$. The $(i,i)$ and $(j,j)$ entries become $0$, and then $(i,j)$ and $(j,i)$ entries become $1$.

### Invertibility

It is important to note that each of the elementary matrices is *invertible*, because each row operation can be undone. The inverse of a permutation matrix $E^{\textrm{perm}}_{ij}$ is itself, because swapping rows $i$ and $j$ twice returns the original matrix. The inverse of a scaling matrix $E^{\textrm{scale}}_i$ is the same matrix with the reciprocal of the scaling factor in the $(i,i)$ entry. Finally, the inverse of an elimination matrix $E^{\textrm{elim}}_{ij}$ is the same matrix, but with the $(i,j)$ entry negated.


In [7]:
# Examples of multiplication by elementary matrices
A = np.array([[0,1,2],[3,4,5],[6,7,8]])
print("A =\n", A)

# P12 swaps row 1 with row 2
P12 = np.array([[0,1,0],[1,0,0],[0,0,1]]) 
B = P12@A
print("B =\n", B)

# E31 eliminates (3,1) entry
# E32 eliminates (3,2) entry
E31 = np.array([[1,0,0],[0,1,0],[-2,0,1]])
E32 = np.array([[1,0,0],[0,1,0],[0,1,1]])
C = E32@E31@B
print("C =\n", C)

# C1 scales row 1
C1 = np.array([[1/3,0,0],[0,1,0],[0,0,1]])
D = C1@C
print("D =\n", D)

A =
 [[0 1 2]
 [3 4 5]
 [6 7 8]]
B =
 [[3 4 5]
 [0 1 2]
 [6 7 8]]
C =
 [[3 4 5]
 [0 1 2]
 [0 0 0]]
D =
 [[1.         1.33333333 1.66666667]
 [0.         1.         2.        ]
 [0.         0.         0.        ]]


In [8]:
# Inverse of P12 is itself
P12_inv = np.linalg.inv(P12)
assert(np.array_equal(P12 @ P12_inv, np.identity(3)))
print("P12 =\n", P12)
print("inverse(P12)=\n", P12_inv)

P12 =
 [[0 1 0]
 [1 0 0]
 [0 0 1]]
inverse(P12)=
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [9]:
# Inverse of elimination matrices
E31_inv = np.linalg.inv(E31)
assert(np.array_equal(E31 @ E31_inv, np.identity(3)))
print("E31=\n", E31)
print("inverse(E31)=\n", E31_inv)
E32_inv = np.linalg.inv(E32)
assert(np.array_equal(E32 @ E32_inv, np.identity(3)))
print("E32=\n", E32)
print("inverse(E32)=\n", E32_inv)

E31=
 [[ 1  0  0]
 [ 0  1  0]
 [-2  0  1]]
inverse(E31)=
 [[ 1. -0. -0.]
 [ 0.  1.  0.]
 [ 2.  0.  1.]]
E32=
 [[1 0 0]
 [0 1 0]
 [0 1 1]]
inverse(E32)=
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  1.]]


In [10]:
# Inverse of C1 should divide row 1 by scale factor
C1_inv = np.linalg.inv(C1)
assert(np.array_equal(C1 @ C1_inv, np.identity(3)))
print("C1=\n", C1)
print("inverse(C1)=\n", C1_inv)

C1=
 [[0.33333333 0.         0.        ]
 [0.         1.         0.        ]
 [0.         0.         1.        ]]
inverse(C1)=
 [[3. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## LU Factorization
#### *Useful for HW4*

A sequence of row operations $E_1$, $...$, $E_k$ can be viewed as a sequence of (invertible) matrix multiplications of the original matrix:

$$ U = E_k E_{k-1} \cdots E_2 E_1 A $$

Here, $A$ is our original $m \times n$ matrix, $U$ is a $m \times n$ matrix in row echelon form, and the $E_i$ matrices are all (invertible) $m \times m$ elementary matrices. We can thus multiply both sides by the inverse of the product of $E_i$ matrices:

$$ A = (E_k E_{k-1} \cdots E_2 E_1)^{-1} U = (E_1^{-1} E_2^{-1} \cdots E_{k-1}^{-1} E_k^{-1}) U $$

Suppose that the sequence of row operations only consists of elimination matrices. **This is the case when we are only concerned with taking $A$ to REF, not RREF.** Elimination matrices and their inverses are *lower triangular* (no nonzero entries above the diagonal), so the product of them is lower triangular as well. We can thus write 

$$A = LU.$$

$L$ is a $m \times m$ lower triangular matrix, and $U$ is a $m \times n$ matrix in row echelon form. If $U$ is also $m \times m$, then it is upper triangular. This is the **LU factorization** of the matrix $A$.

The LU factorization is useful for solving a system of linear equations many times when the coefficient matrix $A$ stays the same while the RHS $\mathbf b$ changes. To solve $A \mathbf x = L U \mathbf x = \mathbf b$, we solve two triangular systems, each using substitution. We first solve $L \mathbf c = \mathbf b$ for $\mathbf c$ using *forward substitution*, since $L$ is lower triangular (equivalent to computing $\mathbf c = L^{-1} \mathbf b$). We then solve $U \mathbf x = \mathbf c$ using back substitution. This is the solution to the original system.

### Other LU factorizations

If the sequence of row operations contains permutations, then they can be combined (multiplied) into one cumulative permutation matrix $P$, and we can write 

$$A = PLU.$$

Lastly, you will see that while $L$ is guaranteed to have 1s along its diagonal, $U$ may not as it is not in reduced row echelon form. We can thus scale each row by the reciprocal of the corresponding pivot entry. Each of those scalings will be captured in a cumulative diagonal scaling matrix $D$, factoring $A$ as 

$$A = PLDU.$$

In this form, $L$ and $U$ are nicely symmetric, with 1s along the diagonal in both matrices.

ps: The process of LU factorization is pretty straightforward for computers as it only involves flipping the last rows over and over. This is why it is implemented by scipy

In [11]:
# LU factorization procedure is in scipy library
import scipy as sp

A = np.array([[1,-2,1,0],[0,2,-8,8],[5,0,-5,10]])
P,L,U = sp.linalg.lu(A)
print(P)
print(L)
print(U)

AttributeError: ignored

In [12]:
# homework 4.4
E1 = np.array([[1,0,0],[-2,1,0],[0,0,1]])
E2 = np.array([[1,0,0],[0,1,0],[-3,0,1]])
E3 = np.array([[1,0,0],[0,1,0],[0,-5/7,1]])
E4 = np.array([[1,0,0],[0,1,0],[0,0,-1/9]])
E5 = np.array([[1,0,0],[0,-1/7,0],[0,0,1]])
E6 = np.array([[1,0,0],[0,1,-1],[0,0,1]])
E7 = np.array([[1,0,-4],[0,1,0],[0,0,1]])
E8 = np.array([[1,-2,0],[0,1,0],[0,0,1]])

In [14]:
# homework 4.4
print("E3@E2@E1")
print(E3@E2@E1)
print("\n\n")

print("E8@E7@E6@E5@E4")
print(E8@E7@E6@E5@E4)
print("\n\n")
E1_inv = np.linalg.inv(E1)
E2_inv = np.linalg.inv(E2)
E3_inv = np.linalg.inv(E3)
E4_inv = np.linalg.inv(E4)
E5_inv = np.linalg.inv(E5)
E6_inv = np.linalg.inv(E6)
E7_inv = np.linalg.inv(E7)
E8_inv = np.linalg.inv(E8)

print("E1_inv@E2_inv@E3_inv")
print(E1_inv@E2_inv@E3_inv)
print("\n\n")
print("E4_inv@E5_inv@E6_inv@E7_inv@E8_inv")
print(E4_inv@E5_inv@E6_inv@E7_inv@E8_inv)
print("\n\n")

E3@E2@E1
[[ 1.          0.          0.        ]
 [-2.          1.          0.        ]
 [-1.57142857 -0.71428571  1.        ]]



E8@E7@E6@E5@E4
[[ 1.          0.28571429  0.22222222]
 [ 0.         -0.14285714  0.11111111]
 [ 0.          0.         -0.11111111]]



E1_inv@E2_inv@E3_inv
[[1.         0.         0.        ]
 [2.         1.         0.        ]
 [3.         0.71428571 1.        ]]



E4_inv@E5_inv@E6_inv@E7_inv@E8_inv
[[ 1.  2.  4.]
 [ 0. -7. -7.]
 [ 0.  0. -9.]]





In [15]:
#  homework 4.5
E1 = np.array([[1,2,3,4], [2,3,4,5], [0,0,6,7], [0,0,7,8]])
E1_inv = np.linalg.inv(E1)
print(E1_inv)

[[-3.  2.  6. -5.]
 [ 2. -1. -5.  4.]
 [ 0.  0. -8.  7.]
 [ 0.  0.  7. -6.]]
